In [3]:
import numpy as np
from numpy.random import choice
from sklearn.decomposition import PCA
import sqlite3
import pandas as pd

import scipy as sp
import sklearn as skl

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.model_selection import cross_validate,KFold
from sklearn import metrics
from sklearn import cross_validation
from sklearn import preprocessing
import csv
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import sklearn.model_selection as ms

In [4]:
engine = sqlite3.connect('CleanData-Apr15')

totaldata_df = pd.read_sql('select * from data', engine)
train_data, test_data= ms.train_test_split(totaldata_df, test_size=0.3, random_state=42)
scaler = StandardScaler()
scaler.fit(train_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']])
train_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']] =scaler.transform(train_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']])
test_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']] =scaler.transform(test_data[['CENTROID_X','CENTROID_Y','DistCBD','pop_density','housing_ratio']])

X_train=train_data.drop(labels=['WIDTH'],axis='columns')
y_train=train_data[['WIDTH']]
X_test=test_data.drop(labels=['WIDTH'],axis='columns')
y_test=test_data[['WIDTH']]


/Users/ilangold/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/ilangold/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/ilangold/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [5]:
#adapted from R code found here: https://github.com/ron1818/PhD_code/blob/master/AdaBoost/myAdaBoostRT.R

def AdaBoostRT(x, y, model, phi, xtest = None, maxIter=100):
    (m, n) = np.shape(x)

    T=maxIter # max iter
    #initialize weight distr and error
    epsilon = np.zeros(T)
    beta = np.zeros(T)
    D = np.zeros(T)
    fit = np.zeros((T, m))
    e = np.zeros((T, m))
    w = np.zeros((T + 1, m))
    w[0,:]= np.repeat(1/m, m)
    models = [0 for t in range(T)]
    t=0
    while (t<(T)):
        idx = np.random.choice(list(range(1, m+1)),  size = (1, m), replace = True, p = w[t, :])
        tth_x=x.iloc[idx[0] - 1,:]
        tth_y=y.iloc[idx[0] - 1,:]
        # call weak learner, CART
        models[t]=model.fit(tth_x,tth_y)
        # getback to hypothesis
        fit[t,:]=model.predict(x)
        # adjusted error for each instance
        y_np = np.transpose(np.asarray(y))
        ARE=np.abs(np.divide((y_np - np.transpose(fit[t,:])),y_np))
        is_ARE_gt_phi = ARE > phi
        is_ARE_le_phi = ARE <= phi
        # calculate error of hypothesis
        epsilon[t] = np.sum(w[t,is_ARE_gt_phi[0]])
        # calculate beta
        beta[t]=epsilon[t] * epsilon[t]
        # update weight vector
        w[t+1,is_ARE_le_phi[0]]=w[t,is_ARE_le_phi[0]]*beta[t]
        w[t+1,is_ARE_gt_phi[0]]=w[t,is_ARE_gt_phi[0]]
        # normalize
        w[t+1,]=np.divide(w[t+1,:],np.sum(w[t+1,:]))
        t=t+1
    # final model
    max_t=t-1
    ww=np.log(np.divide(1,beta[list(range(0, max_t))]))
    aggregatedfit = np.divide((np.matmul(ww , fit[list(range(0, max_t)), :])) , np.sum(ww))
    # predict
    if xtest is None: # have test dataset
        aggregatedpredict= None
    else:
        (mtest, n) = np.shape(xtest)
        prediction = np.zeros((max_t,mtest))
        for t in range(max_t):
            prediction[t,:] = model.predict(xtest)
        aggregatedpredict = np.matmul(ww, prediction) / sum(ww)
    return aggregatedfit, aggregatedpredict

In [9]:
regr = DecisionTreeRegressor(random_state = 314, min_samples_split=68, max_depth=5)

In [15]:
#Cross Validate Paramters
for p in [.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9]:
    boost, pred = AdaBoostRT(X_train, y_train,  model = regr, phi = .8, xtest=X_test, maxIter=100)
    print(pred)
    print('PHI')
    print(p)
    print('err sq and err perc')
    print(np.mean(np.abs((np.array(y_test).ravel() - np.array(pred).ravel()) / np.array(y_test).ravel())) * 100)
    print(np.sqrt(sum(np.square(np.array(pred).ravel() - np.array(y_test).ravel())) / len(y_test)))

[ 3.1100182   2.29974971  6.72125316 ...,  2.29974971  3.1100182
  6.72125316]
PHI
0.01
err sq and err perc
100.044416145
11.2722551643
[ 25.99384634  67.3127274    3.04506384 ...,   3.04506384  67.3127274
  21.8772206 ]
PHI
0.05
err sq and err perc
183.709705388
16.8181901477
[ 3.18766793  3.18766793  3.18766793 ...,  3.18766793  3.18766793
  3.18766793]
PHI
0.1
err sq and err perc
64.1474417418
7.96196964329
[ 20.71405424  20.71405424  20.71405424 ...,  20.71405424  20.71405424
  20.71405424]
PHI
0.2
err sq and err perc
264.844755635
18.8763155216
[ 22.60488845  22.60488845  22.60488845 ...,  22.60488845  22.60488845
  22.60488845]
PHI
0.3
err sq and err perc
230.595495288
14.9861028567
[ 23.58514031  21.18662987  21.18662987 ...,  21.18662987  21.18662987
  21.18662987]
PHI
0.4
err sq and err perc
232.502988882
15.8585097573
[ 23.24256256  23.24256256  23.24256256 ...,  23.24256256  23.24256256
  23.24256256]
PHI
0.5
err sq and err perc
246.955338636
16.0755902091
[  2.79226745  22.